---
title: "Algorithms for Unconstrained Optimization"
author: "Vahram Poghosyan"
date: "2022-01-23"
categories: ["Optimization"]
format:
  html:
    code-fold: true
jupyter: python3
include-after-body:
  text: |
    <script type="application/javascript" src="../../javascript/light-dark.js"></script>
---

# Introduction

***Gradient Descent (GD)*** is a powerful, yet incredibly simple, iterative optimization algorithm. We can think of it as a ***greedy algorithm*** in the setting of continuous optimization. That is, one step of GD is our best attempt at local optimization given limited information about the objective $f(x)$, and having limited computational power at our disposal.

We can further qualify what we mean by *'limited information'* about the objective by introducing a categorization on optimization algorithms – the ***Oracle Access Model (OAM)***. In this model, the objective is abstracted into a black box. For each input $x$, the black box gives the algorithm access to the value of the objective, $f(x)$, and, optionally, global information in the form of higher order behavior such as $\nabla f(x)$, $\nabla^2 f(x)$, etc. GD is what's known as a ***first-order oracle*** because it's only allowed access to $f(x)$ and first-order information in the form of $\nabla f(x)$. 

It's important to note that the OAM is not all-inclusive, there are a number of optimization algorithms, such as ***composite optimization*** algorithms, that utilize structural information about the objective that goes beyond $n$-th order behavior. An example of such an algorithm is ***Proximal Gradient Descent (PGD)*** which, in addition to $f(x)$ and $\nabla f(x)$, also has access to the ***prox operator***: $Prox_{h,\eta}(x)$.

We will cover these composite optimization algorithms in later posts. Many of the composite optimization algorithms, such as PGD, are simple modifications of vanilla GD. The modification done in PGD, for example, make it suitable for constrained optimization.
For now, however, we focus on the case of unconstrained optimization with oracles, particularly on Gradient Descent, in order to develop the key algorithmic intuition.

# The Gradient Descent Algorithm

In this section, we will explore two ideas that give rise to the GD algorithm. As all iterative algorithms, gradient descent relies on an ***initialization step*** and an ***update step***.

## Idea 1 - Greedy Choice of Direction

Let $x$ be the initial iterate, and let the update be given by:

$$x^+ = x + \eta d$$ 

for some directional unit-vector $d$ and ***step-size*** parameter $\eta > 0$.

We base the algorithm on the assumption that the linear approximation of the objective at a the next iterate $x^+$ is a good-enough estimate of its true value at $x^+$. 

That is:

$$f(x^+) = f(x + \eta d) \approx f(x) + \eta \nabla f(x)^T d \ \ \forall d \tag{1.1}$$


Immediately, a locally optimal choice presents itself to us. Since we wish to minimize $f(x)$, it would be wise to insist that the objective at $x^+$ improves or, at least, does not worsen. 

That is, we insist: 

$$f(x^+) \approx f(x) + \eta \nabla f(x)^T d \leq f(x) \tag{1.3}$$

And, since we are greedy in our approach, we wish to make $f(x^+)$ as small as possible. Since, on the RHS, $f(x)$ is fixed and $\eta > 0$, this amounts to minimizing the scaled inner-product $\nabla f(x)^Td$. To that end, we choose $d$ opposite and parallel to the gradient, i.e. $d = - \frac{\nabla f(x)}{||\nabla f(x)||_2}$.

The update step becomes:

$$x^+ = x - \eta \frac{\nabla f(x)}{||\nabla f(x)||_2}$$

By re-labeling, $\eta$ can absorb the normalization constant. This obtains the gradient descent update step as it's often introduced in the textbooks – a step in the negative gradient direction: 

$$x^+ = x - \eta \nabla f(x) \tag{1.4}$$

This makes intuitive sense because the negative gradient direction is the direction in which the objective decreases most. So, it's only natural that the update should take us in this most enticing direction.

## Idea 2 - Greedy Choice of Next Iterate

Instead of defining the update step $x^+ = x + \eta d$ and then choosing the locally optimal direction $d$ greedily, we can choose the update step and the direction, both, in one fell swoop.

Starting from the linear approximation:

$$
f(y) \approx f(x) + \nabla f(x)^T(y - x) \ \ \forall y \tag{2.1}
$$

We can now insist, in a greedy fashion, that the next iterate $x^+$ be the minimizer of the linear approximation. That is, we insist:


$$
x^+ = \arg \min_y f(x) + \nabla f(x)^T(y - x) \tag{2.2}
$$

But the linear approximation is only local, so it would be wise to distrust it for points far away from the current iterate. In this case, since the linear approximation is, in fact, unbounded below, $(2.2)$ would obtain $x^+ = \pm \infty$. To avoid this problem, we introduce a parametrized penalty term that prevents $x^+$ from venturing too far from the current iterate $x$. That is:

$$
x^+ = \arg \min_y f(x) + \nabla f(x)^T(y - x) + \eta ||y - x||_2^2 \tag{2.3}
$$

Now, since the RHS is a a simple quadratic in $y$, it has a unique minimizer which can be found by using the unconstrained optimality condition. This just boils down to taking the gradient of the RHS w.r.t. the optimization variable $y$, setting it to zero, and then solving for the unique root. This procedure obtains: 

$$x^+ = x - \frac{1}{2 \eta} \nabla f(x)$$

By re-labeling, we, once again, get the canonical form of the GD update step as in $(1.3)$ – a step in the negative gradient direction.

# Important Questions in Analysis

Given the ease with which we came up with the algorithm, we should ask ourselves the following questions:

1. Is GD sensitive to initialization?
2. Is GD guaranteed to converge for all step-sizes?
3. How should we choose a step-size that guarantees convergence? 
4. What's the rate of convergence of GD? Does the rate depend on step-size? Does it depend on properties of the objective function?
5. How should we choose a step-size that maximizes convergence rate?

We will shortly explore each of these questions and more. However, before doing so, it's worth taking a bird's eye look at the problem of convex optimization itself. Perhaps the most important question to ask ourselves is this: does gradient descent's convergence rate, for an optimally chosen step-size, give a taxonomy of easier-to-harder problems within the field of convex optimization? The answer, as it turns out, is *yes*.

## Initialization

From this point on, we will limit our discussion to convex objectives in order to eliminate the possibility of strictly ***local optimizers*** (i.e. ***non-global optimizers***) and ***inflection points***, both of which GD gets stuck at if initialized poorly. This ensures the only ***stationary points***, points at which $\nabla f(x) = 0$ and the GD update makes no further progress, are global minimizers. On such convex functions, as we will soon discover, GD has a convergence guarantee for all step-sizes independently of initialization.

## Fixed Step-Size GD

To kickstart our analysis of GD, we consider the fixed step-size algorithm first. Let's take two quintessential convex problems in $\mathbb{R}$, $f(x) = x^2$ and $h(x) = |x|$, and analyze GD's performance on them.

### Simple Analysis of Fixed Step-Size GD

---

First, let's run the algorithm on $h(x) = |x|$ for $x \in \mathbb{R}$:

Since $|x|$ is non-differentiable at $x = 0$, the gradient has a discontinuity at $x = 0$. Non-differentiability, such as this, will eventually force us to introduce the notion of ***sub-gradients***, but for now we can get away with it simply by avoiding the gradient's behavior at $0$. So:

$$
h'(x) = 
\begin{cases} 
\begin{aligned} 
-1 \ &\textrm{if $x < 0$} \\ 
1 \ &\textrm{if $x > 0$} 
\end{aligned}
\end{cases}
$$

Then, for a fixed $\eta > 0$, the update step is:

$$x^+ = x \pm \eta$$

where the sign of $\eta$ depends on where the previous iterate, $x$, falls within the domain $(-\infty, 0) \cup (0, \infty)$.

---

Now, consider $f(x) = x^2$ for $x \in \mathbb{R}$:


The gradient of $f(x) = x^2$ is $f'(x) = 2x$, which means the fixed step-size update is:


$$x^+ = x - 2\eta x$$


---

Note that $x^* = 0$ is the unique optimizer of both $f(x)$ and $h(x)$. With this in mind, there are two key observations to make. 

The first is that, for $x$ far away from $x^* = 0$, the update, $2\eta x$, is large (in magnitude). So, if the iterate is far from the optimizer, GD makes fast progress towards it. 

The second observation is that, as $x \rightarrow x^*$, the update becomes small in magnitude. So, as the iterate comes close to the optimizer, GD takes smaller and smaller steps which converge to $0$ in a summable way. This means, we can get the sub-optimality $f(x) - f(x^*)$ to be $\epsilon$-arbitrarily small for *any* fixed step-size $\eta$.

Neither of these observations hold for GD on $h(x) = |x|$ since the update $\eta$ is fixed regardless of the Euclidean distance between $x$ and $x^* = 0$. In particular, this means GD is *not* fast for $x$ far away from $x^*$ and *does not* slow down as $x$ nears $x^*$. Arbitrary accuracy is impossible with a fixed step-size, since the iterates eventually cycle between $x^T - \eta$ and $x^T + \eta$ where $x^T$ is the last unique iterate – that is $x^T \in (-\eta, \eta)$. The sub-optimality, consequently, also cycles between two values which depend on the choice of $\eta$. This is to say that the sub-optimality cannot be $\epsilon$-arbitrary small for a fixed choice of $\eta$. To be clear, there is still convergence but it's slow and not arbitrarily accurate. Arbitrary accuracy for such problems as this can only be achieved by choosing a sequence of diminishing step-sizes $\{ \eta_t \}_{t=1}^T$ which reduce magnitude of the update since the gradient, itself, is constant. Of course, this sequence must be chosen with care since it's possible to *'run out of steam'*, so to speak, before reaching the optimizer. The precise criterion is $\eta_t \rightarrow 0$ as $t \rightarrow \infty$ s.t. $\sum_t^\infty \eta_t = \infty$.

We say GD on $f(x) = x^2$ enjoys the ***self-tuning property***, whereas GD on $h(x) = |x|$ does not. This speaks to the fact that the self-tuning is a property of the objective functions, rather than GD itself. 

As an overview of the theory we will soon develop, functions *like* $x^2$ (or, more generally, any quadratic in $\mathbb{R}^n$) will all have the self-tuning property while functions *like* $|x|$ will not. This is what ends up introducing the taxonomy of easier-to-harder convex optimization problems mentioned in the previous section. What it means, precisely, to be *like* $x^2$ or $|x|$ will be made rigorous in the next few sections.

# Smoothness and Strong Convexity

As we saw above, gradient descent with a fixed step-size behaved much better on $f(x) = x^2$ than on $h(x) = |x|$. Since both of these problems are convex, $x^2$ must have additional properties not shared by $|x|$ that make it more amenable to optimization by GD. These properties turn out to be ***smoothness*** and ***strong convexity***. We will see that these properties provide insight into choosing the best fixed-step size which guarantees faster convergence of GD.

We start by asking ourselves what makes the two quintessential functions $f(x) = x^2$ and $h(x) = |x|$ different from one another. Since the GD update step relies on the gradient, it helps thinking in terms of the differences of the gradients instead of the objective functions themselves.

The first difference of note is that $|x|$ has a discontinuity at $x = 0$ that's not present in $x^2$. At a point of discontinuity the gradient experiences an abrupt jump. So, in general, jumps in the gradient must pose a problem for GD. 

The second difference of note is that $|x|$ is flat compared to $x^2$. In flat regions, the gradient is constant. So, in general, constant regions in the gradient must pose a problem for GD. 

Both of these scenarios can be ruled out with a [***Lipschitz condition***](https://en.wikipedia.org/wiki/Lipschitz_continuity) on the gradient. Lipschitz conditions are both regularity conditions, as well as, growth conditions. So, they can rule out abrupt jumps and ensure there's, at least, some change in the gradient.

We are ready to define the two properties mentioned in the beginning of this section.

> **Smoothness:** &nbsp; We say a function $f(x)$ is ***M-smooth*** if its gradient is ***M-Lipschitz***. That is, if:
>
> $$\exists M > 0 \ \ s.t. \ \ ||\nabla f(x) - \nabla f(y)||_2 \leq M||x-y||_2 \ \ \forall x,y$$


This is a universal upper-bound on the change in gradient which rules out jumps.

> **Strong Convexity:** &nbsp; We say a function $f(x)$ is ***m-strongly-convex*** if:
>
> $$\exists m > 0 \ \ s.t. \ \ ||\nabla f(x) - \nabla f(y)||_2 \geq m||x-y||_2 \ \ \forall x,y$$


This is a universal lower-bound on the change in gradient which rules out the possibility of a constant gradient.

In particular, an $M$-smooth, and $m$-strongly-convex function $f(x)$ satisfies:

$$m||x-y||_2 \leq ||\nabla f(x) - \nabla f(y)||_2 \leq M||x-y||_2 \ \ \forall x,y \tag{3.1}$$

For a twice-differentiable function, there's a more compact way to express these properties using the hessian which makes use of an ordering on matrices introduced by matrix [***definiteness***](https://en.wikipedia.org/wiki/Definite_matrix). After all, Lipschitzness of the gradient, which is nothing but a restriction on its growth, is a statement about the hessian.

$$||\nabla f(x) - \nabla f(y)||_2 \leq M||x-y||_2 \ \ \forall x,y$$
$$\iff$$
$$||\nabla^2 f(x)||_2 \leq M \ \ \forall x$$
$$\iff$$
$$\nabla^2 f(x) \preceq  MI \ \ \forall x \tag{4.1}$$

The first equivalence is by the [***Mean Value Theorem***](https://en.wikipedia.org/wiki/Mean_value_theorem) and the second follows from the definition of ***matrix norm***.

Line $(3.1)$ should be read as *'the maximum eigenvalue of the hessian $\nabla^2 f(x)$ is $M$'*. 

By a symmetric argument, we also have: 

$$\nabla^2 f(x) \succeq mI \ \ \forall x \tag{4.2}$$

Which should be read as *'the minimum eigenvalue of the hessian $\nabla^2 f(x)$ is $m$'*.

Together, $(4.1)$ and $(4.2)$ give the analog of $(3.1)$ for twice-differentiable $M$-smooth and $m$-strongly-convex functions: 

$$mI \preceq \nabla^2 f(x) \preceq MI \ \ \forall x \tag{3.2}$$

Since the hessian represents the curvature of the function, $(3.2)$ is a two-sided bound on the curvature of $f(x)$. So, we see that smoothness and strong convexity also regulate function shape itself. The lower-bound rules out flatness, while the upper-bound rules out discontinuities like corners and cusps.

## Self-Tuning Property

For a convex function that's $M$-smooth and $m$-strongly-convex we have $(3.1)$ which, as a reminder, is:

$$m||x-y||_2 \leq ||\nabla f(x) - \nabla f(y)||_2 \leq M||x-y||_2 \ \ \forall x,y$$

Fixing iterate $x$, and replacing $y$ with the optimizer $x^*$ we have: 

$$m||x-x^*||_2 \leq ||\nabla f(x) - \nabla f(x^*)||_2 \leq M||x-x^*||_2$$

Since $x^*$ is an optimizer $\nabla f(x^*) = 0$, so the above becomes: 

$$m||x-x^*||_2 \leq ||\nabla f(x)||_2 \leq M||x-x^*||_2$$

The first inequality says that the magnitude of the gradient is *at least* a constant multiple of the distance from the optimizer. The second inequality says that the magnitude of the gradient is *at most* a constant multiple of the distance from the optimizer. So, the gradient is large for $x$ far from $x^*$ and gets smaller as $x \rightarrow x^*$. Since the GD update depends on the magnitude of the gradient, this ensures GD has the self-tuning property. So, smoothness and strong-convexity were, indeed, the ideas needed to encapsulate the self-tuning property.

## Quadratic Bounds

Smoothness and strong-convexity, should they hold for a given convex function, give a universal quadratic point-wise upper and lower-bound, respectively, on the function. This is what it means to say that the function is *like* a quadratic. In a sense, all we're saying is that a function is *tightly* asymptotically bounded by a quadratic at every point. That is, at any given point, the function should neither grow slower nor faster than quadratically.

To construct the upper and lower-bounds, we use the following two lemmas:

> **Lemma 1:** &nbsp; If $f$ is convex and $L$-Lipschitz then $g(x) = \frac{L}{2}x^Tx - f(x)$ is convex.

> **Lemma 2:** &nbsp; If $f$ is $m$-strongly-convex then $g(x) = f(x) - \frac{m}{2}x^Tx$ is convex.

Both lemmas are statements of comparative convexity in disguise. ***Lemma 1*** says that $\frac{L}{2}x^Tx$ is more convex than $f$, whereas ***Lemma 2*** says that $f$ is more convex than $\frac{m}{2}x^Tx$. 

It's not difficult to see how these lemmas can assist in sandwiching $f$ between an upper-bound that's more convex and a lower-bound that's less convex.

The bounds themselves come from the quadratic approximation of $f$ as:

$$f(y) \approx f(x) + \nabla f(x)^T(y-x) + \frac{1}{2}(y-x)^T\nabla^2 f(x)(y-x)\ \ \forall y$$

By replacing the hessian with its bounds $mI$ and $MI$ and using the above lemmas we obtain the two bounds as:

$$f(y) \geq f(x) + \nabla f(x)^T(y-x) + \frac{m}{2}||y-x||_2^2 \ \ \forall y$$ 
$$\textrm{and} \tag {5.1}$$
$$f(y) \leq f(x) + \nabla f(x)^T(y-x) + \frac{M}{2}||y-x||_2^2 \ \ \forall y$$

## The Optimal Fixed Step-Size

We already showed that an $M$-smooth and $m$-strongly-convex function enjoys the advantage of self-tuning. But, without knowing how to choose the step-size, we can still cause GD to make slow progress or even diverge. 

After all, gradient descent is based on a local linear approximation of the objective. If we take big steps, we are counting on the linear approximation to be a good-enough estimate far from the current iterate. This may be too optimistic, in which case GD will diverge. Being too pessimistic, however, is also not good. While taking small steps won't cause GD to diverge, it will make GD painfully slow... Slow to the point of making it worthless in practice.

Luckily, the quadratic upper-bound in $(5.1)$ informs our choice of step-size both in terms of a convergence guarantee and in terms of optimality. First, let's develop the convergence guarantee. 

By plugging the update step $x^+ = x - \eta \nabla f(x)$ as $y$ into the upper-bound in $(5.1)$ we obtain:

$$f(x^+) \leq f(x) + \eta(1-\frac{M}{2}\eta)||\nabla f(x)||_2^2 \tag{5.2}$$

As before, it would be wise to insist $f(x^+) \leq f(x)$, which gives the convergence interval as $0 < \eta < \frac{2}{M}$. 

Here, it helps to consider the simple example of quadratics.

---

**Example 1:**

Consider the quadratic form in $\mathbb{R}$ given by $f(x) = \frac{1}{2}M x^2$ where $x, M \in \mathbb{R}$. Here we can think of $M$ as the only, and therefore the largest, eigenvalue of the $1 \times 1$ matrix $[M]$.

Its GD update step looks like: 

$$x^+ = x - \eta M x = (1 - \eta M)x$$

Which, by recursion from iteration $T$ down to the initial iteration, gives: 

$$x^T = (1- \eta M)^Tx_0$$

Then, since $x^* = 0$, convergence is guaranteed by ensuring $|1 - \eta M| < 1$ or, equivalently, $0 < \eta < \frac{2}{M}$ as desired.

This generalizes to higher dimensions as we shall see in the following example.

---

**Example 2:**

Consider the quadratic form in $\mathbb{R}^n$ given by $f(x) = \frac{1}{2}x^TQx$ where $x \in \mathbb{R}^n$ and $Q \succeq 0$.

Its GD update step looks like: 


$$x^+ = x - \eta Qx = (I - \eta Q)x$$ 

Which, by recursion from iteration $T$ down to the initial iteration, gives: 

$$x^T = \underbrace{(I - \eta Q)\ldots(I - \eta Q)}_{\text{$T$ times}}x_0$$


The eigenvalues $\tilde \lambda_i$ of the matrix $I - \eta Q$ are related to the eigenvalues $\lambda_i$ of $Q$ according to:

$$\tilde \lambda_i = 1 - \eta \lambda_i$$

Hence, if $\lambda_{min} = m$ and $\lambda_{max} = M$, then $\tilde \lambda_{min} = 1 - \eta M$ and $\tilde \lambda_{max} =1 - \eta m$.

The eigenvalues of $I - \eta Q$ act on the current iterate by scaling it. So, in order to ensure convergence to $0$, we need $\tilde \lambda_i \in (-1,1) \ \ \forall i$. Or, equivalently:

$$\tilde \lambda_{min} > -1$$
$$\textrm{and}$$
$$\tilde \lambda_{max} < 1$$

Both of these together give us $0 < \eta < \frac{2}{M}$ as desired.

---

But $0 < \eta < \frac{2}{M}$ is an interval, not a greedy choice. It's just a condition that guarantees convergence. By making the greedy choice we can find the optimal step-size within this interval.

Since the RHS of $(5.2)$ is strongly convex, the quadratic upper-bound is guaranteed to have a unique minimizer. 

The idea is similar to other instances of making a greedy choice we've seen so far. Since the function value is upper-bounded by this quadratic, minimizing this upper-bound gives the best guarantee of smallness for the function value available to us without access to higher order information about the objective. So, the greedy choice for the next iterate is:

$$
x^+ = \arg \min_y f(x) + \nabla f(x)^T(y-x) + \frac{M}{2}||y-x||_2^2
$$

As always, minimizing a quadratic is easy. After going through the motions we obtain:

$$x^+ = x - \frac{1}{M}\nabla f(x)$$

So, the optimal fixed step-size is $\eta = \frac{1}{M}$.

We will see this idea of minimizing a quadratic approximation of the objective, instead of the objective itself, repeat itself when we get to ***Newton's Method (NM)***. However, NM is a second-oder oracle which has access to $\nabla^2 f(x)$, and hence the quadratic approximation at all points in the domain of the objective is accessible to NM. The remarkable thing about $M$-smoothness and $m$-strong-convexity is that they give gradient descent, a first-order oracle, access to *universal* quadratic bounds which eliminates the need to know $\nabla^2 f(x)$. These upper-bounds are, as we saw, what inform GD's choice of step-size which ends up guaranteeing convergence. 

## Convergence Rate

As shown above, the theoretical best fixed step-size for an $M$-smooth objective $f(x)$ is $\eta = \frac{1}{M}$. With this choice of step-size, we can derive convergence guarantees for GD as well as its convergence rate. 

### $M$-Smooth Objectives

Fixing the current iterate as $x$, and plugging in $x^+ = x - \frac{1}{M} \nabla f(x)$ into the upper-bound in $(5.1)$, we obtain the quadratic upper-bound on the next iterate in terms of the magnitude of the gradient: 


$$f(x^+) \leq f(x) - \frac{1}{2M}||\nabla f(x)||_2^2 \tag{5.3}$$

Furthermore, since the underlying assumption throughout this post is that the objective is convex, we have a linear lower-bound $\forall y$, and particularly at the optimizer $y = x^*$, as:

$$f(x^*) \geq f(x) + \nabla f(x)^T(x^* - x) \tag{5.4}$$ 

By reversing $(5.4)$ and adding it to $(5.3)$ we get:

$$f(x^+) \leq f(x^*) + \nabla f(x)^T(x - x^*) - \frac{1}{2M}||\nabla f(x)||_2^2$$

With a bit of algebraic finessing, we can bring the above to the form:

$$f(x^+) \leq f(x^*) + \frac{M}{2}\left[ ||x-x^*||_2^2 - ||x - \frac{1}{M}\nabla f(x) - x^*||_2^2\right]$$

But $x - \frac{1}{M}\nabla f(x) = x^+$, so we have:

$$f(x^+) - f(x^*) \leq \frac{M}{2}\left[ ||x-x^*||_2^2 - ||x^+ - x^*||_2^2\right]$$

We recognize, $||x^+ - x^*||_2^2$ as the sub-optimality of the next iterate, and $||x - x^*||_2^2$ as the sub-optimality of the previous iterate. When both sides are summed over $T$ iterations, the RHS sum telescopes and we're left with:

$$\sum_{t = 0}^{T-1} (f(x_{t+1}) - f(x_t)) \leq \frac{M}{2}||x_0 - x^*||_2^2$$

The LHS is the sum of sub-optimalities across all $T$ iterations, and the RHS is a quantity that's proportional to the initial condition. Taking the average error across all iterations, we get:

$$\frac{1}{T} \sum_{t = 0}^{T-1} (f(x_{t+1}) - f(x_t)) \leq \frac{M}{2T}||x_0 - x^*||_2^2$$

But, we know that the algorithm with fixed step-size $\eta = \frac{1}{M}$ has the descent property since $0 < \frac{1}{M} < \frac{2}{M}$. So, the last sub-optimality $f(x_{T}) - f(x^*)$ must be the smallest. In particular, it must be smaller than the average. So, we have: 

$$f(x_{T}) - f(x^*) \leq \frac{M}{2T}||x_0 - x^*||_2^2 \tag{5.5}$$

So, the error gets better with more iterations and, conversely, gets worse the larger $M$, a measure of how abruptly the gradient changes anywhere on its domain, is. 

Note that $M$, as well as $||x_0 - x^*||_2^2$ are fixed in $(5.5)$. So, the convergence rate of GD on an $M$-smooth objective is $O(\frac{1}{T})$.

### $M$-Smooth and $m$-Strongly-Convex Objectives

The situation is drastically better if, in addition to $M$-smoothness, we also have $m$-strong-convexity. Not only do we get a much faster convergence rate, we also guarantee convergence in the iterates themselves. Note that, so far, we've discussed sub-optimality in objective values only. That is, the only convergence guarantee we've seen so far is $f(x^T) \rightarrow f(x^*)$ as $T \rightarrow \infty$. Sometimes more is needed, we may actually want convergence of the iterates themselves. That is, we may want $x^T \rightarrow x^*$ as $T \rightarrow \infty$? Since strong convexity guarantees the existence of a unique optimizer $x^*$, we can discuss this type of sub-optimality for $m$-strongly-convex objectives.

In this scenario, we have the analog of $(5.3)$ for the quadratic lower-bound. 

Just as $x - \frac{1}{M} \nabla f(x)$ minimized the quadratic upper-bound, $x - \frac{1}{m} \nabla f(x)$ minimizes the quadratic lower-bound. Plugging it into the lower-bound, we get $(5.3)$'s analog as:


$$f(y) \geq f(x) - \frac{1}{2m}||\nabla f(x)||_2^2 \ \ \forall y \tag{5.6}$$


::: {.callout-tip title="Note" appearance="minimal" collapse="false"}
This result is stronger than its analog $(5.3)$ because it holds $\forall y$ as opposed to $(5.3)$ which is only guaranteed to hold at the minimizer $x^+ = x - \frac{1}{M}\nabla f(x)$. This is expected because $(5.6)$ is the result of minimizing a universal lower-bound as opposed to $(5.3)$ which is the result of minimizing a universal upper-bound.
:::

Since $(5.6)$ holds $\forall y$, it holds, in particular, at the optimizer $y=x^*$:

$$f(x^*) \geq f(x) - \frac{1}{2m}||\nabla f(x)||_2^2 \tag{5.7}$$

We can now solve for $||\nabla f(x)||_2^2$ in $(5.7)$ and plug the result into $(5.3)$. From $(5.7)$, we get:

$$||\nabla f(x)||_2^2 \geq 2m(f(x)-f(x^*))$$

Which, when plugged into $(5.3)$, gives:

$$f(x^+) - f(x^*) \leq \left(1-\frac{m}{M}\right)(f(x)-f(x^*))$$

We recognize the LHS as the sub-optimality at the next iteration, and the RHS as the sub-optimality at the current iteration. Recursion from iteration $T$ down to the initial iteration, gives: 

$$f(x_T) - f(x^*) \leq  \left(1-\frac{m}{M}\right)^T (f(x_0) - f(x^*)) \tag{5.8}$$

And, since $m \leq M$ and both strictly positive, $0 < \frac{m}{M} \leq 1$ which guarantees convergence. 

Note that $m$, $M$, and the initial sub-optimality $f(x_0) - f(x^*)$ are fixed quantities in $(5.8)$. So, the convergence rate of GD on a smooth and strongly-convex objective is $O(c^{-T})$ for the constant $c^{-1} = 1 - \frac{m}{M}$. That is, the error decreases *exponentially* in the number of iterations. However, historically, mathematicians were concerned with the logarithm of the error, rather than the error itself, and hence this type of convergence is known as ***linear convergence***.

As promised, we also have convergence of the iterates themselves. From the quadratic lower-bound in $(5.1)$ we can derive an upper-bound on $||x - x^*||_2$ as follows. As before, letting $y = x^*$ in the quadratic lower-bound gives us:

$$f(x^*) \geq f(x) + \nabla f(x)^T(x^* - x) + \frac{m}{2}||x^* - x||_2^2$$

But, by the [***Cauchy-Schwarz Inequality***](https://en.wikipedia.org/wiki/Cauchy%E2%80%93Schwarz_inequality), we further have: 

$$f(x^*) \geq f(x) - ||\nabla f(x)||_2||(x^* - x)||_2 + \frac{m}{2}||x^* - x||_2^2$$

But, since $f(x^*) \leq f(x)$ by the optimality of $x^*$, we must have:

$$- ||\nabla f(x)||_2||(x^* - x)||_2 + \frac{m}{2}||x^* - x||_2^2 \leq 0$$

From which it follows that:

$$||x - x^*||_2 \leq \frac{2}{m}||\nabla f(x)||_2 \tag{5.9}$$

As GD converges to $f(x^*)$, $\nabla f(x) \rightarrow \nabla f(x^*) = 0$ where the last equality is by optimality of $x^*$. So, $x \rightarrow x^*$.

### Affine Invariance

Not only does $(5.8)$ give the rate of convergence of GD, it also predicts its performance on objectives with roughly spherical vs roughly elliptical level-sets. These are the level-sets of what's referred to as ***badly*** and ***well conditioned*** objectives respectively. 

To say that an objective is $M$-smooth and $m$-strongly convex is to say $(3.2)$ which, as a reminder, is: 

$$mI \preceq \nabla^2 f(x) \preceq MI \ \ \forall x$$

This implies that all the eigenvalues of the hessian are bounded between $m$ and $M$. The eigenvalues of the hessian represent the stretch of the level sets in the principal directions. So, to say that $\frac{m}{M} \approx 1$ is to say that $m \approx M$, which means that the level-sets are not more or less stretched in any particular direction. This implies that the level-sets are roughly spherical. As we can see from $(5.8)$, GD converges quite fast in such cases since $\left( 1 - \frac{m}{M} \right)$, the factor of decrease, is small. The opposite is true in the case when the level sets are elongated.

This brings us to an important property called ***affine invariance*** which GD lacks. Simply put, an affine transformation of the input space (i.e. a mere change of coordinates/basis) may affect GD's performance drastically.

It helps to look at an example where the objective is a simple quadratic in $\mathbb{R}^n$.

---

Take the quadratic objective $f(x) = \frac{1}{2}x^TQx$ where $x \in \mathbb{R}^n$ is a coordinate vector in the standard basis. Now, consider the change of coordinates from the standard basis to a basis $Z$ given by $Az = x$ where $A$ is a matrix whose column vectors are the basis vectors in $Z$. 

The key observation is that we can choose $A$ in such a way as to make the objective in the new coordinate system have more or less elliptical level-sets which would affect GD's performance.

First, let's come up with the same quadratic in $Z$-coordinates. 

$$f(x) = f(Az) = \frac{1}{2}(Az)^TQAz = z^TA^TQAz$$

Let $\tilde Q = A^TQA$ and define the quadratic in $Z$-coordinates as $\tilde f(z) = \frac{1}{2}z^T \tilde Q z$ so that $\tilde f(z) = f(x)$ for all $z$ in the $Z$-coordinates corresponding to $x$ in the standard basis. In particular, optimizing in either coordinate system yields the same result in terms of an optimum, that is $\tilde f(z^*) = f(x^*)$.

The GD factor of decrease on $\tilde f$ is $\left(1 - \frac{\tilde m}{\tilde M} \right)$, where $\tilde m = \lambda_{\min}(\tilde Q)$ and $\tilde M = \lambda_{\max}(\tilde Q)$. But $\left(1 - \frac{\tilde m}{\tilde M} \right)$ is under no obligation to be equal to $\left(1 - \frac{m}{M} \right)$, the GD factor of decrease in the standard basis, proving that GD is *not* affine invariant.

Another perspective on affine invariance comes from comparing the GD update steps in both spaces.

The GD update in the standard basis is: 

$$x^+ = x - \eta \nabla f(x) = x - \eta Qx$$

Whereas the GD update in $Z$-coordinates is:

$$z^+ = z - \eta \nabla \tilde f(z) = z - \eta \tilde Qz$$

To go from $Z$-coordinates back to the standard basis, we apply $A$ to the LHS which necessitates its application to the RHS as well. We obtain: 

$$Az^+ = Az - \eta A\tilde Qz = x - \eta AA^TQx$$

Which is *not* the same as $x - \eta Qx$. So, although $Az = x$ the linear relationship breaks down for the next iterates produced by GD (that is $Az^+ \ne x^+$). So, doing a step of GD in the standard basis it's not the same as doing a step of GD in $Z$-coordinates (up to a change of basis by $A$). So, gradient descent is doing something radically different in the $Z$-coordinates compared to what it does in the standard basis.

---

#### Best Affine Transformation

A natural question to ask, at this point, is which choice of $A$ makes GD perform faster on a quadratic objective? 

Algebraically, the best we can hope for is $A$ s.t. $\tilde m \approx \tilde M$. One way we can accomplish this is by forcing *all* of the eigenvalues of $\tilde Q$ to be the same. Particularly, letting them all be $1$ by enforcing $\tilde Q = A^TQA = I$ works. 

Since $Q \succeq 0$, it has an eigendecomposition as $Q = PDP^T$ where $D$ is diagonal and $P$ is orthonormal. Then its matrix square root exists and is given by $Q^{-1/2} = PD^{-1/2}P^T$.
Letting $A = Q^{-1/2}$ we, indeed, have:

$$
\begin{aligned}
A^TQA &= (PD^{-1/2}P^T)^TPDP^TPD^{-1/2}P^T \\ 
&= PD^{-1/2}DD^{-1/2}P^T \\
&= PP^T \\
&= I \\
\end{aligned}$$

Where we have used the fact that $P^TP = PP^T = I$ since $P$ is orthonormal, and the fact that diagonal matrices are raised to a power simply by raising their diagonal entries to that power.

Geometrically, the choice of $\tilde Q = I$ forces the level-sets to be spherical. A level-set of $f(x) = \frac{1}{2}x^TQx$ in the standard coordinate system, i.e. an ellipse in $\mathbb{R}^n$, is given by $x^TQx = c$ for some constant $c$ which has absorbed $\frac{1}{2}$. In the $Z$-coordinates, the same level-set is given by $\tilde f(z) = z^T\tilde Qz = c$. If $\tilde Q = I$, as is the case for the choice $A = Q^{-1/2}$, then the level-set in the $Z$-coordinates becomes $z^Tz = c$ which is, indeed, a sphere in $\mathbb{R}^n$.

In conclusion, if the objective is quadratic, we can improve GD's convergence rate by applying the above change of basis. If the objective is not quadratic, we may still assume that it's locally quadratic. This allows us to apply the same idea to non-quadratic objectives, but, since it requires coming up with a different matrix $A$ at each iteration, the payoff from this procedure may not be worth it. So, we explore other ways of accelerating the performance of gradient descent on badly conditioned objectives by developing ***Accelerated Gradient Descent (AGD)*** which will be explained shortly.

# Variable Step-Size GD

While smoothness gives the theoretical best step-size $\eta = \frac{1}{M}$, for most problems it can be quite difficult, or impossible, to come up with the smoothness parameter $M$. So, while informative, the discussion we've had so far has been largely theoretical. In practice, there are subroutines such as ***Exact Line Search (ELS)*** or ***Backtracking Line Search (BTLS)*** which choose an appropriate step-size $\eta_t$ at each iteration. These subroutines can also be modified and used with other optimization algorithms. For example, BTLS is the state of the art way of choosing a step-size in Newton's and ***Quasi-Newton's Methods***.

## Exact Line Search (ELS)

Let's go back to the general iterative update step $x^+ = x + \eta d$.

By restricting the objective in the direction of the update $d$ we can find the optimal step-size, $\eta^*$, at each iteration by solving the following one-dimensional, unconstrained optimization problem in $\eta$:

$$\eta^* = \arg \min_{\eta} f(x + \eta d) \tag{6.1}$$

We proceed with the optimization by defining the restriction of $f$ in the direction $d$ as $\phi(\eta) := f(x + \eta d)$. Then, we can use the chain-rule to find the stationary point $\eta^*$ for which $\nabla \phi(\eta^*) = 0$.

By chain-rule:

$$\nabla \phi(\eta) = \nabla f(x + \eta d)^T d \tag{6.2}$$

In the case of GD, $d = -\nabla f(x)$, so we have: 

$$\nabla \phi(\eta) = \nabla f(x - \eta \nabla f(x))^T (-\nabla f(x)) \tag{6.3}$$

An interesting geometric consequence of this is that GD with ELS takes perpendicular steps that end at a point of tangency with a level-set. Setting $\nabla \phi(\eta) = 0$ to find the optimal step-size obtains $\eta^*$ s.t. $\nabla f(x - \eta^* \nabla f(x))$ is perpendicular to $- \nabla f(x)$. In other words, GD with ELS goes in the negative gradient direction until the gradient at $x - \eta^* \nabla f(x)$ is perpendicular to the gradient at the current iterate $x$. At the next iteration, GD will take a step in the $- \nabla f(x - \eta^* \nabla f(x))$ direction which is still perpendicular to $- \nabla f(x)$. This means, at each iteration, GD takes a step that's perpendicular to the step it took in the previous iteration. Furthermore, since gradients are always perpendicular to level-sets, the new iterate $x - \eta^* \nabla f(x)$ is a point of tangency with the level-set at that point.

Although this subroutine is very natural, it may be infeasible to solve an optimization problem (even a one-dimensional one) at each iteration. Hence, we introduce backtracking line search.

## Backtracking Line Search (BTLS)

As we know a convex objective $f$ is always lower-bounded by its linear approximation. That is:

$$f(y) \geq f(x) + \nabla f(x)^T(y - x) \ \ \forall y$$

Plugging $x^+ = x + \eta d$ into the above lower-bound obtains:

$$f(x^+) \geq f(x) + \eta \nabla f(x)^T d$$

So, the greatest possible reduction in value from $f(x)$ to $f(x^+)$ we can hope for is $\eta \nabla f(x)^T d$ which, recall, is non-positive by a choice of $d$ that guarantees descent (such as $d = -\nabla f(x)$ in gradient descent). Unless the objective is linear, in which case the above linear approximation holds with equality, we can not hope to achieve the full $\eta \nabla f(x)^T d$ reduction in value. This is just a consequence of convexity and is, therefore, also the case when using exact line search. 

The idea behind backtracking line search is to ensure we achieve, at least, a fraction of this maximum reduction in value by introducing a parameter $0 < \alpha < 1$.

Since $f(x) + \eta \nabla f(x)^T d$, the linear underestimate of $f(x^+)$, is the tangent line to $f$ at $x$ in the direction $d$, $f(x) + \alpha \eta \nabla f(x)^T d$ (for $0 < \alpha < 1$) is a secant line of $f$ at $x$ in the direction $d$. Setting $\alpha = \frac{1}{2}$, a typical choice in practice, and finding the *largest* $\eta$ s.t. $f(x^+) = (x + \eta d)$ is still below this secant line ensures we get approximately half of the maximum reduction in value $\eta \nabla f(x)^T d$. 

This is exactly what the BTLS subroutine, detailed below, tries to achieve:

### The BTLS Subroutine

1. The BTLS subroutine takes as input $0 < \alpha < 1$, and $0 < \beta < 1$. 

2. While $f(x + \eta d) > f(x) + \alpha \eta \nabla f(x)^T d$, it reduces $\eta$ by the update $\eta \leftarrow$ $\beta \eta$. 

### Convergence Guarantees of GD with BTLS

Gradient descent with backtracking line search has a promising convergence guarantee for convex objectives that are also $M$-smooth. 

> **Convergence of GD with BTLS:** &nbsp; If $f$ is $M$-smooth and convex then the step-size given by BTLS is s.t.
> 
> $\eta_{BTLS} \geq \frac{\beta}{M}$. Furthermore, $f(x^+) - f(x) \leq \frac{\alpha \beta}{M}||\nabla f(x)||_2^2$

So, compared to the theoretical best step-size $\eta = \frac{1}{M}$, which may not be accessible to us, we have $\frac{\beta}{M} \leq \eta_{BTLS} < \frac{1}{M}$. So, $\beta_{BTLS}$ is less than the optimal step-size but, interestingly, it still keeps $M$ in view despite the latter being unknown to us. 

Furthermore, for an $M$-smooth objective, the final sub-optimality after $T$ iterations can be found as:

$$f(x_T) - f(x^*) \leq \frac{M}{2T \alpha \beta}||x_0 - x^*||_2^2 \tag{7.1}$$

Which is only a constant factor $\alpha \beta$ worse than GD with the theoretical best fixed step-size. So, it's still $O(1/T)$.

For an $M$-smooth objective that's also $m$-strongly convex, we have: 

$$f(x_T) - f(x^*) \leq \left (1 - \min \left\{ 2m\alpha, \frac{2\alpha\beta m}{M} \right \} \right )^T||x_0 - x^*||_2^2 \tag{7.2}$$

Which is, again, comparable to GD with the theoretical best fixed step-size.

# Theoretical Error Bounds on First Order Oracles

As we saw the convergence rates of GD were upper-bounded by $O\left (\frac{1}{T} \right )$ and $O\left (\left(1 - \frac{m}{M} \right)^T \right)$ for $M$ and $m$ conditioned objectives. We also mentioned that GD can be accelerated, which begs the question: how much can we improve the error? 

Turns out, there are asymptotic lower-bounds on the error produced by *any* first-order oracle. So, just as the error of GD cannot be worse that its asymptotic upper-bounds, it cannot be better than its asymptotic lower-bounds. 

First, we define a first order oracle more generally as any iterative algorithm of the form: 

$$x_{t+1} \in x_t + span \{ \nabla f(x_t), \nabla f(x_{t-1}), ..., \nabla f(x_0)\}$$

Note that GD falls in this category as it only uses $\nabla f(x_t)$ which is, indeed, in the span.

For any such algorithm, the lower-bounds on the error are given as the following two-part theorem.

> **First-Order Oracle Error Lower-Bounds:** &nbsp; For any first-order Oracle:
> 
> 1. $\exists$ a convex, $M$-smooth function $f$ for which the error is $\Omega \left (\frac{1}{T^2} \right )$
> 2. $\exists$ an $M$-smooth, $m$-strongly-convex function $f$ for which the error is $\Omega \left (\left (1-\sqrt{\frac{m}{M}} \right )^T \right )$ 

::: {.callout-tip title="Note" appearance="minimal" collapse="false"}
Since $0 < \frac{m}{M} \leq 1$, its square root is larger. Hence, $1 - \sqrt{\frac{m}{M}}$ is smaller than $1 - \frac{m}{M}$. So $\left (1-\sqrt{\frac{m}{M}} \right )^T$, the lower bound, indeed grows slower than $\left ( 1 - \frac{m}{M} \right )^T$, the upper-bound.
:::

Any first-order oracle, including Accelerated GD, cannot hope for error that's asymptotically better than these lower-bounds. In fact, we shall see that AGD achieves these lower-bounds exactly, thereby closing the first-order oracle error gap.